In [2]:
!pip install -q openai
!pip install langchain langchain-openai -q
!pip install faiss-cpu -q
!pip install langchain_community -q
!pip install tiktoken -q

In [3]:
from openai import OpenAI
MODEL = "gpt-4o-mini-2024-07-18"
client = OpenAI(api_key = '')

In [14]:
content = """
문서 1:
북극곰은 바다 얼음에 의존하여 물범을 사냥하기 때문에 기후 변화에 직접적인 영향을 받고 있습니다. 지구 온난화로 인해 바다 얼음이 매년 더 일찍 녹고 늦게 형성되면서,
북극곰이 사냥할 수 있는 시간이 줄어들고 있습니다. 이는 북극곰의 신체 상태 악화, 새끼의 생존율 저하, 그리고 일부 경우에는 사망률 증가로 이어졌습니다.

문서 2:
해빙 서식지의 상실은 북극곰 개체군에 가장 큰 위협 중 하나입니다. 얼음이 녹으면서 북극곰은 먹이를 찾기 위해 더 먼 거리를 이동해야 하며 이로 인해 에너지 소모가 증가합니다.
이로 인해 영양실조와 생식 성공률이 감소하여 종을 더욱 위험에 빠뜨릴 수 있습니다.

문서 3:
기후 변화는 북극곰의 사냥터뿐만 아니라 보금자리 구역에도 영향을 미치고 있습니다. 더 따뜻한 기온과 불안정한 눈 조건으로 인해
임신한 암컷이 적합한 보금자리 지역을 찾는 것이 더 어려워지고 있으며, 이는 새끼의 출생과 생존에 매우 중요합니다.
이는 바다 얼음 손실로 이미 어려움을 겪고 있는 북극곰 개체군에 추가적인 위험 요소가 되고 있습니다.

문서 4:
전통적인 식량 공급원이 점점 더 어려워지면서 일부 북극곰은 인간의 배설물을 먹거나 바다새와 그 알을 잡아먹는 모습이 관찰되었습니다.
이러한 행동은 단기적인 안도감을 제공할 수 있지만, 혹독한 북극 환경에서 장기적으로 생존하는 데 필수적인 물개로부터 얻는 고지방 식단을 대체하지는 못합니다.

문서 5:
최근 연구에 따르면 온실 가스 배출의 현재 추세가 계속된다면 북극곰은 이 세기 안에 멸종에 직면할 수 있습니다.
보존 노력은 전 세계 배출량을 줄이고 중요한 북극곰 서식지를 보호하는 데 집중하고 있지만 기후가 현재 속도로 계속 따뜻해진다면 이러한 노력만으로는 충분하지 않을 수 있습니다.
"""

with open('documents_ko.txt', 'w') as f:
    f.write(content)

In [6]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

In [25]:
# OpenAI API 키 설정
import os
os.environ["OPENAI_API_KEY"] = ''

# 1. 문서 로드
loader = TextLoader('documents_ko.txt')
documents = loader.load()

# 2. 문서를 임베딩 및 인덱싱
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# 3. Fine_tuned GPT 모델을 사용한 LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)

# 4. 문서 검색 개수 조정 (최상위 5개의 유사한 백터를 반환하여 검색)
retriever = vector_store.as_retriever(search_kwargs={"k":5})

# 5. RAG 구현
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 6. 질문에 대한 답변생성
query = "기후변화가 북극곰에게 끼치는 영향이 뭐에요?"
result = qa_chain.invoke({'query': query})

# 7. 결과 출력
print(result["result"])

기후 변화는 북극곰에게 여러 가지 심각한 영향을 미치고 있습니다. 

1. **서식지 손실**: 바다 얼음이 매년 더 일찍 녹고 늦게 형성되면서 북극곰의 사냥터가 줄어들고 있습니다. 이는 북극곰이 물범을 사냥할 수 있는 시간이 감소하게 만듭니다.

2. **영양 상태 악화**: 얼음이 녹으면서 북극곰은 먹이를 찾기 위해 더 먼 거리를 이동해야 하며, 이로 인해 에너지 소모가 증가합니다. 결과적으로 영양실조와 생식 성공률이 감소하게 됩니다.

3. **보금자리 문제**: 더 따뜻한 기온과 불안정한 눈 조건으로 인해 임신한 암컷이 적합한 보금자리 지역을 찾는 것이 어려워지고, 이는 새끼의 출생과 생존에 부정적인 영향을 미칩니다.

4. **식량 부족**: 전통적인 식량 공급원이 줄어들면서 일부 북극곰은 인간의 배설물이나 바다새와 그 알을 잡아먹는 행동을 보이고 있습니다. 그러나 이러한 대체 식단은 고지방 식단을 제공하지 못해 장기적인 생존에는 도움이 되지 않습니다.

5. **멸종 위험**: 연구에 따르면 현재 온실 가스 배출 추세가 계속된다면 북극곰은 이 세기 안에 멸종에 직면할 수 있습니다. 보존 노력에도 불구하고 기후 변화가 계속된다면 이러한 노력만으로는 충분하지 않을 수 있습니다. 

이러한 요인들은 북극곰의 생존에 심각한 위협이 되고 있습니다.
